# Exploratory Data Analysis (EDA) on Fuel Blend Properties Dataset

**Goal**: Develop accurate predictive models that can estimate 10 key properties of blended fuels, enabling faster development and optimization of sustainable fuel formulations.

**Dataset Description**:
- `train.csv`: Training data containing blend composition, component properties, and target blend properties
- `test.csv`: Test data with only input features; model must predict the 10 blend properties
- `sample_submission.csv`: Template for submission format

**Column Groups in Train.csv**:
- **Blend Composition (first 5 columns)**: Volume percentages of each of the 5 base components (e.g., *Component1_fraction*).
- **Component Properties (next 50 columns)**: 10 Properties of each component batch (e.g., *Component1_Property1*).
- **Final Blend Properties - Targets (last 10 columns)**: Target properties to predict (e.g., *BlendProperty1*).

**Column Groups in Test.csv**:
- **Id (first column)**: Repesents the unique number for each row (e.g., *1*).
- **Blend Composition (next 5 columns)**: Volume percentages of each of the 5 base components (e.g., *Component1_fraction*).
- **Component Properties (last 50 columns)**: 10 Properties of each component batch (e.g., *Component1_Property1*).

**Evaluation Metric**:
The evaluation metric used is the **Mean Absolute Percentage Error (MAPE)**, defined as:
$$\text{MAPE} = \frac{1}{n} \sum_{t=1}^{n} \left| \frac{y_t - \hat{y}_t}{y_t} \right| \times 100$$

Where:
- $y_t$: Actual value
- $\hat{y}_t$: Predicted value

For reporting purposes, scores are normalized using the formula:
$$\text{Score} = \max(0, 100 - \left(\frac{\text{cost}}{\text{reference cost}}\right) \times 100)$$

## Step 1: Load & Explore the Dataset

In [1]:
import pandas as pd
from pathlib import Path

# Directory of the datasets
data_path = Path('../data')

# Load the raw dataset
train_data = pd.read_csv(data_path / 'train.csv')
test_data = pd.read_csv(data_path / 'test.csv')

# Shape of the datasets
print("Shape of train data: ", train_data.shape)
print("Shape of test data: ", test_data.shape)

Shape of train data:  (2000, 65)
Shape of test data:  (500, 56)


In [2]:
# First 5 rows of the train data
train_data.head(5)

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.489143,0.607589,0.321670,-1.236055,1.601132,1.384662,0.305850,0.193460,0.580374,-0.762738
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-1.257481,-1.475283,-0.437385,-1.402911,0.147941,-1.143244,-0.439171,-1.379041,-1.280989,-0.503625
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,1.784349,0.450467,0.622687,1.375614,-0.428790,1.161616,0.601289,0.872950,0.660000,2.024576
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-0.066422,0.483730,-1.865442,-0.046295,-0.163820,-0.209693,-1.840566,0.300293,-0.351336,-1.551914
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.118913,-1.172398,0.301785,-1.787407,-0.493361,-0.528049,0.286344,-0.265192,0.430513,0.735073


In [3]:
# Information about the dataset
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    2000 non-null   float64
 1   Component2_fraction    2000 non-null   float64
 2   Component3_fraction    2000 non-null   float64
 3   Component4_fraction    2000 non-null   float64
 4   Component5_fraction    2000 non-null   float64
 5   Component1_Property1   2000 non-null   float64
 6   Component2_Property1   2000 non-null   float64
 7   Component3_Property1   2000 non-null   float64
 8   Component4_Property1   2000 non-null   float64
 9   Component5_Property1   2000 non-null   float64
 10  Component1_Property2   2000 non-null   float64
 11  Component2_Property2   2000 non-null   float64
 12  Component3_Property2   2000 non-null   float64
 13  Component4_Property2   2000 non-null   float64
 14  Component5_Property2   2000 non-null   float64
 15  Comp

In [4]:
# Statistical info
train_data.describe(include='all')

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
count,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.18069,0.182910,0.179820,0.342090,0.114490,0.000245,-0.017319,0.001703,-0.004653,-0.018256,...,-0.016879,-0.002076,-0.014351,-0.006068,-0.015249,-0.003497,-0.013568,-0.017236,-0.001507,-0.001795
std,0.16320,0.163704,0.166283,0.141119,0.080219,0.999423,1.006409,0.998859,1.006902,1.009294,...,0.993787,1.004512,0.999360,1.009176,0.986480,1.009126,1.000613,0.998759,1.001096,0.990433
min,0.00000,0.000000,0.000000,0.010000,0.000000,-2.943737,-1.718895,-3.008683,-3.029468,-3.572440,...,-2.550897,-3.079759,-3.041624,-2.835701,-1.730111,-2.808210,-2.994571,-3.621080,-3.292727,-2.476429
25%,0.03000,0.040000,0.020000,0.220000,0.050000,-0.694658,-0.765154,-0.701948,-0.693361,-0.713149,...,-0.766128,-0.735109,-0.624235,-0.783547,-0.683165,-0.697379,-0.622453,-0.725564,-0.702384,-0.733653
50%,0.14000,0.150000,0.140000,0.350000,0.120000,0.011977,-0.030235,0.021335,0.016774,0.194936,...,-0.021089,0.001684,0.146135,-0.028158,-0.250650,-0.011649,0.133470,-0.001548,-0.002604,-0.010459
75%,0.29000,0.300000,0.290000,0.500000,0.180000,0.685717,0.653960,0.673125,0.659227,1.032029,...,0.714763,0.723807,0.727597,0.664659,0.358701,0.695182,0.704130,0.684894,0.706084,0.693839
max,0.50000,0.500000,0.500000,0.500000,0.290000,2.981146,3.051090,2.868901,2.982258,1.032029,...,2.856588,2.769156,1.638646,3.769643,3.600439,3.433292,3.293228,3.340657,3.276199,2.708703


In [5]:
# Get column groups
fraction_cols = [col for col in train_data.columns if 'fraction' in col]
property_cols = [col for col in train_data.columns if 'Property' in col and 'Blend' not in col]
blend_cols = [col for col in train_data.columns if 'BlendProperty' in col]
print(f"Fraction Columns: {len(fraction_cols)}\nProperty Columns: {len(property_cols)}\nBlend Columns: {len(blend_cols)}")

Fraction Columns: 5
Property Columns: 50
Blend Columns: 10


Insights on the dataset:
- No null or duplicate values are present in the dataset
- Only numerical values are present in the dataset
- Volume percentages of each of the 5 base components are in plausible range (0.00-1.00)

## Step 2: Analyze & Visualize the Dataset

#### 2.1 Blend Composition Distribution

In [6]:
## Histogram
import numpy as np
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
from plotly.subplots import make_subplots

# Define a bold color palette (extend as needed)
bold_colors = ['#1f77b4', '#ff7f0e', '#2ca02c',
               '#d62728', '#9467bd', '#8c564b']

# Create subplot grid for histograms + KDEs
fig_hist_fractions = make_subplots(
    rows=2, cols=3,
    subplot_titles=fraction_cols,
    vertical_spacing=0.1
)

# Loop through each component fraction
for i, col in enumerate(fraction_cols):
    row = i // 3 + 1  # Determine subplot row
    col_pos = i % 3 + 1  # Determine subplot column

    data = train_data[col]

    # Add histogram trace with bold color and white borders
    fig_hist_fractions.add_trace(
        go.Histogram(
            x=data,
            name=col,
            nbinsx=30,
            marker=dict(
                color=bold_colors[i % len(bold_colors)],  # Cycle colors
                line=dict(color='white', width=1)  # White bar dividers
            ),
            opacity=0.6
        ),
        row=row, col=col_pos
    )

    # Compute KDE (kernel density estimate)
    kde = gaussian_kde(data)
    x_vals = np.linspace(data.min(), data.max(), 200)
    y_vals = kde(x_vals) * len(data) * (data.max() - data.min()) / 30  # Scale to match hist height

    # Add KDE line over histogram
    fig_hist_fractions.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='lines',
            line=dict(color='black', width=2),
            name=f"{col} KDE"
        ),
        row=row, col=col_pos
    )

# Final layout tweaks
fig_hist_fractions.update_layout(
    title_text="Distribution of Component Fractions with KDE Overlay",
    showlegend=False,
    height=700
)

fig_hist_fractions.show()

Plot Insights:
- Component1–3:
    - **High frequency at 0 and 0.5**: Indicates these components are often either absent or at maximum allowed proportion.
    - **Relatively uniform in between**: Mild variation in mid-ranges.

- Component4:
    - **Skewed toward higher values, especially 0.5**: Often present in large amounts—likely a dominant or essential component in most blends.

- Component5:
    - **Concentrated around 0.1–0.2, with a drop-off after 0.3**: Typically added in small to moderate quantities, rarely exceeds 0.3.

In [7]:
import plotly.express as px

# Sum each component's contribution across all rows
total_fractions = train_data[fraction_cols].sum(axis=0)

# Prepare data for pie chart
pie_data = total_fractions.reset_index()
pie_data.columns = ['Component', 'TotalFraction']

# Create and show the pie chart using Plotly
fig = px.pie(
    pie_data,
    values='TotalFraction',
    names='Component',
    title='Overall Contribution of Each Component',
    hover_name='Component'
)

colors = ['#0B4689','#08306B','#155FA0','#3E92CC', '#1E77B5',]
fig.update_traces(textposition='inside',
                  textinfo='percent',
                  marker=dict(colors=colors, line=dict(color='#FFFFFF', width=3)))
fig.update_layout(showlegend=True)
fig.show()

#### 2.2 Blend Property Distribution

In [ ]:
# Bold color palette for blend properties (extend as needed)
blend_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Create 2x5 subplot grid for 10 blend properties
fig_hist_blend = make_subplots(
    rows=2, cols=5,
    subplot_titles=blend_cols,
    vertical_spacing=0.1
)

# Loop through each blend property
for i, col in enumerate(blend_cols):
    row = i // 5 + 1
    col_pos = i % 5 + 1

    data = train_data[col].dropna()

    # Add histogram with bold color and white separator lines
    fig_hist_blend.add_trace(
        go.Histogram(
            x=data,
            name=col,
            nbinsx=30,
            marker=dict(
                color=blend_colors[i % len(blend_colors)],
                line=dict(color='white', width=1)
            ),
            opacity=0.6
        ),
        row=row, col=col_pos
    )

    # KDE overlay
    kde = gaussian_kde(data)
    x_vals = np.linspace(data.min(), data.max(), 200)
    y_vals = kde(x_vals) * len(data) * (data.max() - data.min()) / 30  # Scale to match histogram

    fig_hist_blend.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='lines',
            line=dict(color='black', width=2),
            name=f"{col} KDE"
        ),
        row=row, col=col_pos
    )

# Update layout
fig_hist_blend.update_layout(
    title_text="Distribution of Blend Properties with KDE Overlay",
    showlegend=False,
    height=700
)

fig_hist_blend.show()

In [9]:
# Selecting last 10 blend property columns
data = train_data.iloc[:, 55:]

# Melt the DataFrame to long format for Plotly
melted_data = data.melt(var_name='Property', value_name='Value')

# Create the violin plot
fig = px.violin(melted_data, x='Property', y='Value', color='Property', box=True, points='all', color_discrete_sequence=px.colors.qualitative.Bold)

# Update layout for aesthetics
fig.update_layout(
    showlegend=False,
    title='Distribution of Blend Properties',
    yaxis_title='Value',
    xaxis_title=None,
    xaxis_tickangle=45
)

fig.show()

In [10]:
## Boxplot
fig_box_blend = go.Figure()

for col in blend_cols:
    fig_box_blend.add_trace(
        go.Box(y=train_data[col], name=col, boxpoints='outliers')
    )

fig_box_blend.update_layout(
    showlegend=False,
    title="Boxplots of Blend Properties",
    yaxis_title="Value",
    height=500
)

###

### component property

###

In [25]:
# Create 10x5 subplot grid for 50 component properties
fig_hist_blend = make_subplots(
    rows=10, cols=5,
    subplot_titles=property_cols,
    vertical_spacing=0.05,  # reduced for tighter layout
    horizontal_spacing=0.03
)

bold_colors = ['#1f77b4', '#ff7f0e', '#2ca02c',
               '#d62728', '#9467bd', '#8c564b']

# Loop through each property
for i, col in enumerate(property_cols):
    row = i // 5 + 1
    col_pos = i % 5 + 1

    data = train_data[col].dropna()

    if len(data) == 0:
        continue

    # Histogram
    fig_hist_blend.add_trace(
        go.Histogram(
            x=data,
            nbinsx=30,
            marker=dict(
                color=bold_colors[i % len(bold_colors)],
                line=dict(color='white', width=1)
            ),
            opacity=0.6,
            showlegend=False
        ),
        row=row, col=col_pos
    )

    # KDE overlay
    try:
        kde = gaussian_kde(data)
        x_vals = np.linspace(data.min(), data.max(), 200)
        bin_width = (data.max() - data.min()) / 30
        y_vals = kde(x_vals) * len(data) * bin_width  # Scale to match histogram

        fig_hist_blend.add_trace(
            go.Scatter(
                x=x_vals,
                y=y_vals,
                mode='lines',
                line=dict(color='black', width=2),
                showlegend=False
            ),
            row=row, col=col_pos
        )
    except Exception as e:
        print(f"KDE failed for {col}: {e}")

# Update layout
fig_hist_blend.update_layout(
    title_text="Distribution of Component Properties with KDE Overlay",
    height=3000,  # increased to accommodate all 50 plots
    width=1200,
    margin=dict(t=100),
    template="plotly_white"
)

fig_hist_blend.update_yaxes(title_text='Count', showgrid=False)
fig_hist_blend.update_xaxes(showgrid=False)

fig_hist_blend.show()

#### 2.3 Correlation Heatmaps and Pairplot

In [11]:
# Component Fraction vs Blend Properties
subset_cols = fraction_cols + blend_cols
corr_matrix = train_data[subset_cols].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='blues',
    zmid=0,
    text=np.round(corr_matrix.values, 2),
    texttemplate="%{text}",
))

fig.update_layout(
    title="Correlation Heatmap - Fractions vs Blend Properties",
    width=1200,
    height=800,
    xaxis_tickangle=-45
)

In [26]:
# Full features
corr_matrix = train_data.corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='blues',
    zmid=0,
    text=np.round(corr_matrix.values, 2),
    texttemplate="%{text}",
))

fig.update_layout(
    title="Correlation Heatmap - All Features",
    width=1700,
    height=1300,
    xaxis_tickangle=-45
)

Key Observations from Your Plots:

- **Component Fractions**: Component 4 shows extreme concentration at 0.5, while Components 1-3 are more uniformly distributed. Component 5 has a bell-shaped distribution.
- **Component Contributions**: Component 4 dominates at 34.2%, suggesting it might be a primary base fuel.
- **Blend Properties Distribution**: Most properties follow normal distributions, but some show skewness or multiple modes.
- **Correlations**: Strong correlations exist between certain component fractions and blend properties, indicating predictable relationships.
- **Component Properties**: Some properties show non-normal distributions with outliers.